# Ames Housing

Predict the value of SalePrice variable

## 1. Import Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import os

def get_dataset(data_dir, train_fn, test_fn):

    train_path = os.path.join(data_dir, train_fn)
    test_path = os.path.join(data_dir, test_fn)

    return pd.read_csv(train_path), pd.read_csv(test_path)

train_set, test_set = get_dataset('dataset', 'train.csv', 'test.csv')

In [ ]:
train_set.head(5)

In [ ]:
train_set.info()

In [ ]:
test_set.info()

Store Id column and drop them from training and test set (not necessary for predictions)

In [ ]:
train_ID = train_set['Id']
test_ID = test_set['Id']

X_train = train_set.drop('Id', axis=1)
X_test = test_set.drop('Id', axis=1)

## 2. Analyzing Data

Render **histogram of dependent variable** `SalePrice`

In [ ]:
def plot_density_hist(x):
    sns.histplot(x, stat='density', kde=False, bins=50)
    sns.kdeplot(x, cut=3, color='crimson')

plot_density_hist(X_train['SalePrice'])

- Deviate from normal distribution
- Positive skewed
- Show peakedness (high *kurtosis*)

In [ ]:
print(f'Skewness: {X_train.SalePrice.skew():.2f}')
print(f'Kurtosis: {X_train.SalePrice.kurt():.2f}')

Let's see how variables are correlated using a **Correlation Matrix (heatmap)**.

In [ ]:
corrmat = X_train.corr()
f, ax = plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=.8, square=True)

Notice bright white squares. These are strong linear correlations between nearing variables. The correlation is so strong that it could indicate that multicollinearity. The biggest perpetrators are `TotalBsmtSF` with `1stFlrSF`, `GarageCars` with `GarageArea`, `GrLivArea` with `TotRmsAbvGrd`, and `YearBuilt` with `GarageYrBlt`.

Also note that the strong correlations between certain variables are our dependent variable, SalePrice. This includes `OverallQual`, `GrLivArea`, `GarageCars`, `GarageArea`, `TotalBsmtSF`, `1stFlrSF`, and to some extent `YearBuilt`, `FullBath`, and a few others.

**Zoomed correlation matrix**

In [ ]:
k = 10 # number of variables for heatmap
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(X_train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

Here we can see which variables have the strongest linear correlation to `SalePrice`. From the previous heatmap we know that both pairs `(GarageCars, GarageArea)` and `(TotalBsmtSF, 1stFlrSF)` are strongly related. Therefore, we can ignore one from each pair to avoid multicollinearity. Let's keep `GarageCars` and `TotalBsmtSF` since they have a slightly stronger correlation to `SalePrice`.

We can also see that `GrLivArea` and `TotRmsAbvGrd` have a strong correlation, suggesting mulitcollinearity. Perhaps we can drop `TotalRmsAbvGrd` as well. 

**Scatter plots between `SalePrice` and correlated variables**

In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(X_train[cols], size=2.5)
plt.show()

## 3. Data Processing

#### **Outliers**

Documentation suggests there may be outliers present in the training data

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = X_train['GrLivArea'], y = X_train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

Remove two values in the bottom right

In [ ]:
#Deleting outliers 
X_train = X_train.drop(X_train[(X_train['GrLivArea']>4000) & (X_train['SalePrice']<300000)].index)

#Check the graphic again
fig, ax = plt.subplots()
ax.scatter(X_train['GrLivArea'], X_train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

Careful deleting too many outliers because they may show up in the test set. We want the models to be robust enough to handle them.

`SalePrice` is the dependent variable (what we need to predict) so let's do some analysis on it

In [ ]:
def normal(mean, std, color="black"):
    x = np.linspace(mean-4*std, mean+4*std)
    p = stats.norm.pdf(x, mean, std)
    z = plt.plot(x, p, color, linewidth=2)

plot_density_hist(X_train['SalePrice'])

# get fitted parameters used by the function
(mu, sigma) = stats.norm.fit(X_train['SalePrice'])
print(f'\n mu = {mu:.2f} and sigma = {sigma:.2f}\n')

normal(mu, sigma)

# now plot the distribution
plt.legend([f'Normal dist. ($\mu=$ {mu:.2f} and $\sigma=$ {sigma:.2f}'], loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

# also get the QQ-plot
fig = plt.figure()
res = stats.probplot(X_train['SalePrice'], plot=plt)
plt.show()

Target variable is right skewed. Linear models love normally distributed data, so we need to transform this variable to make it more normally distributed.

#### **Log-Transformation of the target variable**

In [ ]:
# use np.log1p which applies log(1+x) to all elements of the column
X_train['SalePrice'] = np.log1p(X_train['SalePrice'])

plot_density_hist(X_train['SalePrice'])

# Get the fitted parameters used by the function
(mu, sigma) = stats.norm.fit(X_train['SalePrice'])
print(f'\n mu = {mu:.2f} and sigma = {sigma:.2f}\n')

# normal(mu, sigma)

#Now plot the distribution
plt.legend([f'Normal dist. ($\mu=$ {mu:.2f} and $\sigma=$ {sigma:.2f} )'],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(X_train['SalePrice'], plot=plt)
plt.show()

Much better

### **Features engineering**

Drop unecessary and problem (multicollinearity) features from training and test set

In [ ]:
y_train = X_train['SalePrice']
X_train = X_train.drop(['SalePrice', '1stFlrSF', 'GarageArea', 'TotRmsAbvGrd', 'GarageYrBlt'], axis=1)
X_test = X_test.drop(['1stFlrSF', 'GarageArea', 'TotRmsAbvGrd', 'GarageYrBlt'], axis=1)

In [ ]:
X_train.shape, X_test.shape

#### **Missing Data**

Important questions when thinking about missing data:
- How prevalent is the missing data?
- Is missing data random or does it have a pattern?

In [ ]:
all_data = pd.concat([X_train, X_test]).reset_index(drop=True)
total = all_data.isnull().sum().sort_values(ascending=False)
percent = (all_data.isnull().sum()/all_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(34)

There are elements missing from both training and test sets. We'll have to fill them in if possible. Note that we don't want to introduce bias to the test set by filling them in with any value. If the documentation states that a missing value represents something (like None or 0) then we will fill both sets with those values. Otherwise, we'll have to find the most common or median value within the training set and fill the test set with those values. If values are missing from a feature in the test set but no in the training set, we are left with no other option besides filling it in with whatever we can.

Descriptions for the following variables states that NA means:
- `PoolQC`: no pool
- `MiscFeatures`: no misc features
- `Alley`: no alley
- `Fence`: no fence
- `FireplaceQu`: no fireplace
- `GarageType`, `GarageFinish`, `GarageQual`, `GarageCond`: no garage
- `BsmtQual`, `BsmtCond`, `BsmtExposure`, `BsmtFinType1`, `BsmtFinType2`: no basement
- `MasVnrType`: no masonry veneer

In [ ]:
for col in ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 
    'GarageCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'MasVnrType',
    'MSSubClass']:
    X_train[col] = X_train[col].fillna('None')
    X_test[col] = X_test[col].fillna('None')

- `GarageYrBlt`, `GarageArea`, `GarageCars`: replace missing data with 0 (since No garage means no cars in such garage)
- `BsmtFinSF1`, `BsmtFinSF2`, `BsmtUnSF`, `TotalBsmtSF`, `BsmtFullBath`, `BsmtHalfBath`: missing values are likely zero for having no basement
- `MasVnrArea`: likely 0 for no masonry veneer

In [ ]:
for col in ['GarageCars', 'BsmtFinSF1', 'BsmtFinSF2', 
    'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'MasVnrArea']:
    X_train[col] = X_train[col].fillna(0)
    X_test[col] = X_test[col].fillna(0)

Fill missing values with most frequent category for:
- `MSZoning`
- `Electrical`
- `KitchenQual`
- `Exterior1st`
- `Exterior2nd`
- `SaleType`

In [ ]:
for col in ['Electrical', 'KitchenQual', 'Exterior1st', 'Exterior2nd', 'SaleType', 'MSZoning']:
    X_train[col] = X_train[col].fillna(X_train[col].mode()[0])
    X_test[col] = X_test[col].fillna(X_test[col].mode()[0])

`Utilities`: This feature won't help in predictive modeling so we can safely remove it (every instance in the training set is in the same category). 

In [ ]:
X_train = X_train.drop('Utilities', axis=1)
X_test = X_test.drop('Utilities', axis=1)

`Function` feature with value NaN refers to "Typical"

In [ ]:
X_train["Functional"] = X_train["Functional"].fillna("Typ")
X_test["Functional"] = X_test["Functional"].fillna("Typ")

`LotFrontage`: since lots in the same neighborhood are likely to have similar sizes, fill with median LotFrontage of neighborhood

In [ ]:
# find median LotFrontage for a neighborhood
lot_med = X_train.groupby('Neighborhood')['LotFrontage'].median()

# Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
X_train['LotFrontage'] = X_train.groupby('Neighborhood')['LotFrontage'].transform(
    lambda x: x.fillna(x.median()))

# Fill missing values in X_test with medians from each neighborhood in X
for i, nan in enumerate(X_test['LotFrontage'].isnull()):
    if nan:
        neighborhood = X_test.loc[i, 'Neighborhood']
        X_test.loc[i, 'LotFrontage'] = lot_med[neighborhood]

Check for any remaining missing values in X

In [ ]:
# Check remaining missing values if any 
all_data_na = ((X_train.isnull().sum() + X_test.isnull().sum()) / (len(X_train) + len(X_test))) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

We good

**Transforming some numerical variables that are really categorical**

In [ ]:
#MSSubClass=The building class
X_train['MSSubClass'] = X_train['MSSubClass'].apply(str)

#Changing OverallCond into a categorical variable
X_train['OverallCond'] = X_train['OverallCond'].astype(str)

#Year and month sold are transformed into categorical features.
X_train['YrSold'] = X_train['YrSold'].astype(str)
X_train['MoSold'] = X_train['MoSold'].astype(str)

In [ ]:
#MSSubClass=The building class
X_test['MSSubClass'] = X_test['MSSubClass'].apply(str)

#Changing OverallCond into a categorical variable
X_test['OverallCond'] = X_test['OverallCond'].astype(str)

#Year and month sold are transformed into categorical features.
X_test['YrSold'] = X_test['YrSold'].astype(str)
X_test['MoSold'] = X_test['MoSold'].astype(str)

**Label Encoding some categorical variables that may contain information in their ordering set**

In [ ]:
from sklearn.preprocessing import LabelEncoder

def label_encode(X):
    cols = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
            'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
            'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
            'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
            'YrSold', 'MoSold']
    # process columns, apply LabelEncoder to categorical features
    for c in cols:
        lbl = LabelEncoder()
        lbl.fit(list(X[c].values))
        X[c] = lbl.transform(list(X[c].values))

    print(X.shape)

label_encode(X_train)
label_encode(X_test)


**Skewed Features**

In [ ]:
numeric_feats = X_train.dtypes[X_train.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = X_train[numeric_feats].apply(lambda x: stats.skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

There are some heavily skewed features in the training set. They probably exist in the test set as well.

**Box Cox Transformation of (highly) skewed features**

We use the scipy function boxcox1p which computes the Box-Cox transformation of $1+x$

Note that setting $\lambda=0$ is equivalent to log1p used above for the target variable

In [ ]:
def correct_skewness(X):
    numeric_feats = X.dtypes[X.dtypes != "object"].index

    # Check the skew of all numerical features
    skewed_feats = X[numeric_feats].apply(lambda x: stats.skew(x.dropna())).sort_values(ascending=False)
    skewness_df = pd.DataFrame({'Skew' :skewed_feats})

    skewness = skewness_df[abs(skewness_df) > 0.75]
    print(f"Correcting {skewness.shape[0]} numerical features to Box Cox transform")

    from scipy.special import boxcox1p
    skewed_features = skewness.index
    lam = 0.15
    for f in skewed_features:
        #X[f] += 1
        X[f] = boxcox1p(X[f], lam)
        
    #X[skewed_features] = np.log1p(X[skewed_features])

correct_skewness(X_train)
correct_skewness(X_test)

**One-hot encoding features**

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_attr = X_train.select_dtypes(include='object').columns
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe_cols_train = pd.DataFrame(ohe.fit_transform(X_train[cat_attr]))
ohe_cols_test = pd.DataFrame(ohe.transform(X_test[cat_attr]))

# One-hot encoding removed index; put it back
ohe_cols_train.index = X_train.index
ohe_cols_test.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
numeric_X_train = X_train.drop(cat_attr, axis=1)
numeric_X_test = X_test.drop(cat_attr, axis=1)

# Add one-hot encoded columns to numerical features
X_train = pd.concat([numeric_X_train, ohe_cols_train], axis=1)
X_test = pd.concat([numeric_X_test, ohe_cols_test], axis=1)

X_train.shape, X_test.shape

## 4. Modeling

For scoring

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

k, kfold = 5, StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def rmsle_cv(model):
    kf = kfold.get_n_splits(X_train.values)
    rmse = np.sqrt(-cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=kf))
    return rmse

**Lasso**

In [ ]:
from sklearn.linear_model import Lasso

lasso = Pipeline([
    ('scaler', RobustScaler()),
    ('model', Lasso(random_state=42))
])

lasso.fit(X_train, y_train)
score = rmsle_cv(lasso)
print(f'Lasso: {score.mean():.4f}  ({score.std():.4f})')

**ElasticNet**

In [ ]:
from sklearn.linear_model import ElasticNet

en = Pipeline([
    ('scaler', RobustScaler()), 
    ('model', ElasticNet(random_state=42))
])

en.fit(X_train, y_train)
score = rmsle_cv(en)
print(f'ElasticNet: {score.mean():.4f}  ({score.std():.4f})')

**Kernel Ridge**

In [ ]:
from sklearn.kernel_ridge import KernelRidge

kr = KernelRidge()
kr.fit(X_train, y_train)
score = rmsle_cv(kr)
print(f'Kernel Ridge: {score.mean():.4f}  ({score.std():.4f})')

**XGBoost**

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42, nthread=-1)
xgb.fit(X_train, y_train)
score = rmsle_cv(xgb)
print(f'XGBoost: {score.mean():.4f}  ({score.std():.4f})')

**Gradient Boosting**

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# huber loss makes it robust to outliers
gb = GradientBoostingRegressor(loss='huber', random_state=42)
gb.fit(X_train, y_train)
score = rmsle_cv(gb)
print(f'Gradient Boosting: {score.mean():.4f}  ({score.std():.4f})')

**LightGBM**

In [ ]:
from lightgbm import LGBMRegressor

lgb = LGBMRegressor()
lgb.fit(X_train, y_train)
score = rmsle_cv(lgb)
print(f'LightGB: {score.mean():.4f}  ({score.std():.4f})')

## 5. Hyperparameter Search

In [ ]:
from skopt import BayesSearchCV, space

def optimize(model, param_grid):
    opt = BayesSearchCV(
        model,
        param_grid,
        n_iter=32,
        random_state=42,
        cv=5
    )
    opt.fit(X_train, y_train)
    return opt

**Lasso**

In [ ]:
lasso_params = {
    'model__alpha': space.Real(0, 1, prior='uniform'),
}

opt_lasso = optimize(lasso, lasso_params)
best_lasso = opt_lasso.best_estimator_
opt_lasso.best_params_

In [ ]:
score = rmsle_cv(best_lasso)
print(f'Optimized Lasso: {score.mean():.4f}  ({score.std():.4f})')

**ElasticNet**

In [ ]:
en_params = {
    'model__alpha': space.Real(0, 1, prior='uniform'),
    'model__l1_ratio': space.Real(0, 1, prior='uniform')
}

opt_en = optimize(en, en_params)
best_en = opt_en.best_estimator_
opt_en.best_params_

In [ ]:
score = rmsle_cv(best_en)
print(f'Optimized ElasticNet: {score.mean():.4f}  ({score.std():.4f})')

**XGBoost**

In [ ]:
xgb_params = {
    'colsample_bytree': space.Real(0.1, 0.5, prior='uniform'),
    'gamma': space.Real(0.01, 0.1, prior='uniform'),
    'learning_rate': space.Real(0.01, 0.1, prior='uniform'),
    'max_depth': space.Integer(1, 4),
    'min_child_weight': space.Real(1, 2, prior='uniform'),
    'n_estimators': space.Integer(100, 2500),
    'reg_alpha': space.Real(0.1, 1, prior='uniform'),
    'reg_lambda': space.Real(0.1, 1, prior='uniform'),
    'subsample': space.Real(0.1, 1, prior='uniform'),
}

opt_xgb = optimize(xgb, xgb_params)
best_xgb = opt_xgb.best_estimator_
opt_xgb.best_params_

In [ ]:
score = rmsle_cv(best_xgb)
print(f'Optimized XGBoost: {score.mean():.4f}  ({score.std():.4f})')

**Kernel Ridge**

In [ ]:
kr_params = {
    'alpha': space.Real(0, 1, prior='uniform'),
    'kernel': space.Categorical(['linear', 'polynomial']),
    'degree': space.Integer(2, 5),
    'coef0': space.Real(2, 3, prior='uniform')
}

opt_kr = optimize(kr, kr_params)
best_kr = opt_kr.best_estimator_
opt_kr.best_params_

In [ ]:
score = rmsle_cv(best_kr)
print(f'Optimized Kernel Ridge: {score.mean():.4f}  ({score.std():.4f})')

**Gradient Boosting**

In [ ]:
gb_params = {
    'n_estimators': space.Integer(2500, 3000),
    'learning_rate': space.Real(0.01, 0.1, prior='uniform'),
    'max_depth': space.Integer(1, 5),
    'max_features': space.Categorical(['sqrt', 'log2']),
    'min_samples_leaf': space.Integer(10, 15),
    'min_samples_split': space.Integer(5, 10)
}

# opt_gb = optimize(gb, gb_params)
# best_gb = opt_gb.best_estimator_
# opt_gb.best_params_

best_gb = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state=42)

In [ ]:
score = rmsle_cv(best_gb)
print(f'Optimized Gradient Boosting: {score.mean():.4f}  ({score.std():.4f})')

**LightGB**

In [ ]:
lgb_params = {
    'num_leaves': space.Integer(2, 6),
    'learning_rate': space.Real(0.01, 0.1, prior='uniform'), 
    'n_estimators': space.Integer(100, 1000),
    'max_bin': space.Integer(50, 80),
    'subsample': space.Real(0.5, 1, prior='uniform'),
    'subsample_freq': space.Integer(2, 5),
    'colsample_bytree': space.Real(0.1, 0.3, prior='uniform'),
    'min_child_samples':space.Integer(2, 6),
    'min_child_weight': space.Integer(9, 12)
}

opt_lgb = optimize(lgb, lgb_params)
best_lgb = opt_lgb.best_estimator_
opt_lgb.best_params_                 

In [ ]:
score = rmsle_cv(best_lgb)
print(f'Optimized LightGB: {score.mean():.4f}  ({score.std():.4f})')

### **Stacking**

**Average base models class** (simplest approach)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
    
    # we define clones of the models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]

        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)
        
        return self
    
    # Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

In [ ]:
averaged_models = AveragingModels(models=[best_lasso, best_en, best_xgb, best_kr, best_gb, best_lgb])
score = rmsle_cv(averaged_models)
print(f'Averaged models: {score.mean():.4f}  ({score.std():.4f})')

Even the simplest approach improves the score. This encourages exploring a more complex stacking method.

## Submission

In [ ]:
averaged_models.fit(X_train, y_train)

# take power of (e+1) to undo the log(x+1) transformation and revert data back to the true house prices
y_pred = np.expm1(averaged_models.predict(X_test))

result = pd.DataFrame()
result['Id'] = test_ID
result['SalePrice'] = y_pred
result.to_csv('predictions.csv', index=False)